# 1. Load in data model
## Description
This grabs an example data format of a [vote event](https://www.popoloproject.com/appendices/examples.html#vote-events) to produce more generalized code to create a visualization of voting behavior.

In [8]:
import polars as pl
import altair as alt
import json
import os

### 1.1. Load data

In [ ]:
# First import data as a VoteEvent type
parent = os.path.abspath("..")
path = os.path.join(parent, "data", "example_vote_event.json")
with open(path, "r") as f:
    votes = json.load(f)

c:\Users\micha\Documents\EsteCon\data\example_vote_event.json


In [ ]:
print(votes)

{'identifier': '17', 'motion': {'text': 'The parliment took an example vote'}, 'start_date': '2005-02-22', 'counts': [{'option': 'yes', 'value': 40}, {'option': 'no', 'value': 85}, {'option': 'abstain', 'value': 5}], 'votes': [{'voter': {'name': 'Voter 1'}, 'option': 'yes'}, {'voter': {'name': 'Voter 2'}, 'option': 'yes'}, {'voter': {'name': 'Voter 3'}, 'option': 'yes'}, {'voter': {'name': 'Voter 4'}, 'option': 'yes'}, {'voter': {'name': 'Voter 5'}, 'option': 'yes'}, {'voter': {'name': 'Voter 6'}, 'option': 'yes'}, {'voter': {'name': 'Voter 7'}, 'option': 'yes'}, {'voter': {'name': 'Voter 8'}, 'option': 'yes'}, {'voter': {'name': 'Voter 9'}, 'option': 'yes'}, {'voter': {'name': 'Voter 10'}, 'option': 'yes'}, {'voter': {'name': 'Voter 11'}, 'option': 'yes'}, {'voter': {'name': 'Voter 12'}, 'option': 'yes'}, {'voter': {'name': 'Voter 13'}, 'option': 'yes'}, {'voter': {'name': 'Voter 14'}, 'option': 'yes'}, {'voter': {'name': 'Voter 15'}, 'option': 'yes'}, {'voter': {'name': 'Voter 16'}, 

In [ ]:
# Then extract it into a vote format
vote_date = votes["start_date"]
votes_df = pl.DataFrame(votes["votes"]).unnest("voter")

### 1.2 - Reformat for plot
I take the data and then create a grid and assign the vote information to a plot.

In [53]:
def reformat_votes(df: pl.DataFrame) -> pl.DataFrame:
    """
    Take format and put it into a format to manually plot values for Altair.

    Input:
        df - pl.DataFrame (130 x 3): A single vote

    Returns (pl.DataFrame) polars dataframe to feed to Altair
    """
    # Create lists to fill for reformatted values
    loc_y = []
    loc_x = []
    option = []
    name = []

    # Fill lists with location data from each
    for y in range(1, 11):
        for x in range(1, 14):
            loc_y.append(y)
            loc_x.append(x)

            # Currently do not sort data
            obs = (y - 1) + (x - 1) * 10
            option.append(df.row(obs)[df.columns.index("option")])
            name.append(df.row(obs)[df.columns.index("name")])

    # Load as data frames
    parliment_df = pl.DataFrame(
        {"x": loc_x, "y": loc_y, "option": option, "name": name}
    )
    return parliment_df

In [57]:
plot_votes_df = reformat_votes(votes_df)

### 1.3 Plot in Altair

In [ ]:
def plot_votes(plot_votes_df):
    """
    Creates a series of Chart elements.

    Input:
        - plot_votes_df (pl.DataFrame) - Data frame from vote event object

    Returns (none): plots a basic altair plot
    """
    # Creates eligibility base layer
    votes_plot = (
        alt.Chart(
            plot_votes_df,
            title=alt.Title(
                f"Votes for Bill {votes['identifier']}",
                subtitle=f"Topic: {votes['motion']['text']}",
            ),
        )
        .mark_point(size=100, filled=True)
        .encode(
            alt.X("x")
            .title(None)
            .axis(domain=False, grid=False, labels=False, ticks=False),
            alt.Y("y")
            .title(None)
            .axis(
                domain=False,
                ticks=False,
                grid=False,
                labels=False,
            ),
            color="option",
        )
    )
    return votes_plot

In [72]:
plot_votes(plot_votes_df)

alt.Chart(...)